# Imports

In [17]:
import pandas as pd; import seaborn as sb; import statsmodels.formula.api as smf; import pingouin;
from statannot import add_stat_annotation; from itertools import combinations; import os
from analysis_helpers import *; import scipy; from scipy import stats; import os; import math

import warnings
warnings.filterwarnings('ignore')

# Load and organize data

In [26]:
whole_exp1_gaze = pd.DataFrame.from_csv('/Users/kirstenziman/Documents/attention-memory-task/code/analysis/Experiment1_Full_Pres_Gaze_Trial_Labeled.csv')
whole_exp2_gaze = pd.DataFrame.from_csv('/Users/kirstenziman/Documents/attention-memory-task/code/analysis/whole_exp2_gaze_best.csv')

# whole_exp1_gaze = pd.DataFrame.from_csv('/Users/kirstenziman/Documents/NEW_DIRECTORY/new_place/attention-memory-task/analyses/exp1_gaze.csv')
# whole_exp2_gaze = pd.DataFrame.from_csv('/Users/kirstenziman/Documents/NEW_DIRECTORY/new_place/attention-memory-task/analyses/exp2_gaze.csv')

In [28]:
whole_exp2_gaze = whole_exp2_gaze[whole_exp2_gaze['Subject']!=13]
whole_exp2_gaze = whole_exp2_gaze[whole_exp2_gaze['Subject']!=27]
whole_exp2_gaze['Subject'].unique()

array(['8_', '15', '1_', '25', '12', '4_', '28', '19', '21', '33', '7_',
       '26', '14', '2_', '16', '27', 23, 18, 29, '13', '5_', '24', '0_',
       '34', '22', '32', '17', '3_', '10', '20'], dtype=object)

In [29]:
whole_exp2_gaze['Subject'] = whole_exp2_gaze['Subject'].astype(str)
whole_exp2_gaze['Subject'] = whole_exp2_gaze['Subject'].map(lambda x: x.rstrip('_'))
whole_exp2_gaze['Subject'] = whole_exp2_gaze['Subject'].astype(float)

## tendency to gaze towards cued side

In [30]:
# load behavioral data
exp1 = pd.DataFrame.from_csv('/Users/kirstenziman/Documents/NEW_DIRECTORY/new_place/attention-memory-task/parsed_data/behavioral_data_exp1.csv')
exp2 = pd.DataFrame.from_csv('/Users/kirstenziman/Documents/NEW_DIRECTORY/new_place/attention-memory-task/parsed_data/behavioral_data_exp2.csv')

In [31]:
# exp2['Subject'] = exp2['Subject'].map({327: 32})

In [32]:
for data,behav,label in zip([whole_exp1_gaze, whole_exp2_gaze],[exp1, exp2],['1','2']):
    
    # Make sure behavioral trials are numbered
    behav.loc[behav['Trial Type']=='Memory','Trial'] = list(range(0,40))*30*8
    behav.loc[behav['Trial Type']=='Presentation','Trial'] = list(range(0,10))*30*8
    
    # group gaze data
    gaze_grp = data.groupby(['Subject','Run','Trial'], as_index=False).mean()
    gaze_grp['Run'] = pd.to_numeric(gaze_grp['Run'])
    
    # add av x coord to each presentation trial in behavioral df
    for s in gaze_grp['Subject'].unique():

        if label=='2' and s == 32:
            for r in gaze_grp[gaze_grp['Subject']==s]['Run'].unique():
                for t in gaze_grp[(gaze_grp['Subject']==s) & (gaze_grp['Run']==r)]['Trial'].unique():

                    if gaze_grp[(gaze_grp['Subject']==s) & (gaze_grp['Run']==r) & (gaze_grp['Trial']==t)].shape[0]>0:

                        behav.loc[(behav['Subject']==327) 
                                     & (behav['Run']==r) 
                                     & (behav['Trial']==t), 'av_x_coord'] =  gaze_grp[(gaze_grp['Subject']==s) 
                                                                                     & (gaze_grp['Run']==r) 
                                                                                     & (gaze_grp['Trial']==t)]['av_x_coord'].item()


        else:
            for r in gaze_grp[gaze_grp['Subject']==s]['Run'].unique():
                for t in gaze_grp[(gaze_grp['Subject']==s) & (gaze_grp['Run']==r)]['Trial'].unique():

                    if gaze_grp[(gaze_grp['Subject']==s) & (gaze_grp['Run']==r) & (gaze_grp['Trial']==t)].shape[0]>0:

                        behav.loc[(behav['Subject']==s) 
                                     & (behav['Run']==r) 
                                     & (behav['Trial']==t), 'av_x_coord'] =  gaze_grp[(gaze_grp['Subject']==s) 
                                                                                     & (gaze_grp['Run']==r) 
                                                                                     & (gaze_grp['Trial']==t)]['av_x_coord'].item()

    # difference from screen center
    behav['gaze_from_center'] = behav['av_x_coord'] - 59.8/2
    
    # negative for opposite facing arrow
    behav['gaze_towards_cued_side'] = behav['gaze_from_center']
    behav.loc[(behav['Cued Side']=='<'),'gaze_towards_cued_side'] = -behav.loc[(behav['Cued Side']=='<')]['gaze_towards_cued_side']
    
    # ttest relative to zero drift! 
    behav_grp = behav.groupby(['Subject'], as_index=False).mean()
    print(scipy.stats.ttest_1samp(behav_grp['gaze_towards_cued_side'],0))

Ttest_1sampResult(statistic=3.10115317032973, pvalue=0.004265738366959131)
Ttest_1sampResult(statistic=2.241202377673304, pvalue=0.03283486220324865)


# Data restricted to gaze between images

In [33]:
def cent_to_deg(centimeters, distance):
    answer = math.degrees(math.atan(centimeters/distance))
    return(answer)

In [34]:
cent_to_deg(3.5, 60)

3.3384705437643523

In [35]:
restricted_concat = {}
for gaze,label in zip([whole_exp1_gaze, whole_exp2_gaze],['exp1','exp2']):

    gaze['gaze_from_center'] = gaze['av_x_coord'] - (59.8/2)
    
    gaze_restricted = gaze[abs(gaze['gaze_from_center'])<3.5]
    gaze_restricted = gaze_restricted.groupby(['Subject','Run','Trial'], as_index=False).mean()
    
    restricted_behavioral = []

    for s in gaze_restricted['Subject'].unique():
        for r in gaze_restricted[gaze_restricted['Subject']==s]['Run'].unique():
            for t in gaze_restricted[(gaze_restricted['Subject']==s) & (gaze_restricted['Run']==r)]['Trial'].unique():
                restricted_behavioral.append(exp1[(exp1['Subject']==s) & (exp1['Run']==int(r)) & (exp1['Trial']==float(t))])
    
    restricted_concat[label] = pd.concat(restricted_behavioral)

# Attended Category vs None Images

In [36]:
for l in restricted_concat:
    d = restricted_concat[l]

    Cats = []
    Novs = []
    for s in d['Subject'].unique():

        Cats.append(d[(d['Subject']==s) & (d['Attention Level'].isin(['Category','Full']))]['Familiarity Rating'].mean())
        Novs.append(d[(d['Subject']==s) & (d['Attention Level']=='None')]['Familiarity Rating'].mean())

    print(l)
    print(scipy.stats.ttest_rel(Cats, Novs))
    print()

exp1
Ttest_relResult(statistic=5.726358851335908, pvalue=3.3826876454571282e-06)

exp2
Ttest_relResult(statistic=4.276206395354509, pvalue=0.0002828683173392469)



# Attended Side vs None Images

In [37]:
for l in restricted_concat.keys():
    d = restricted_concat[l]

    Sides = []
    Novs = []
    
    for s in d['Subject'].unique():

        Sides.append(d[(d['Subject']==s) & (d['Attention Level'].isin(['Full','Side']))]['Familiarity Rating'].mean())
        Novs.append(d[(d['Subject']==s) & (d['Attention Level']=='None')]['Familiarity Rating'].mean())


    print(scipy.stats.ttest_rel(Sides, Novs))
    print()

Ttest_relResult(statistic=6.055704093518778, pvalue=1.3672119469717405e-06)

Ttest_relResult(statistic=4.57812229530863, pvalue=0.0001332146936024731)



# Regression and Plot

In [38]:
def add_gaze(df):

    '''
    input: df containing pres and mem from single run
    output: df with string in 'Attention Level' column in each Memory trial row
    '''

    for index,row in df.iterrows():
        if row['Trial Type']=='Memory':
            mem_image = row['Memory Image']
            for cue in ['Cued ', 'Uncued ']:
                for cat in ['Face', 'Place']:
                    if df.loc[df[cue+cat] == mem_image].shape[0]!=0:
                        df['av_x_coord'][index]=df.loc[df[cue+cat] == mem_image]['av_x_coord']
                        df['Cued Side'][index] = df.loc[df[cue+cat] == mem_image]['Cued Side'].item()

    mem_mask = df['Trial Type']=='Memory'
    df.loc[mem_mask,'av_x_coord'] = df.loc[mem_mask,'av_x_coord'].fillna(np.nan)

    return(df)


In [39]:
#pd.concat(gaze_added_mem).to_csv('gaze_added_mem_exp1.csv')
#gaze_added_mem = pd.DataFrame.from_csv('gaze_added_mem_exp1.csv')

In [40]:
#gaze_mem = pd.concat(gaze_added_mem)

In [41]:
gaze_added_mem = []
gaze_added_mem_dict = {}

for data,label in zip([exp1, exp2],['exp1','exp2']):

    for s in data['Subject'].unique():
        for r in data['Run'].unique():
            gaze_added_mem.append(add_gaze(data[(data['Subject']==s) & (data['Run']==r)]))

    gaze_mem = pd.concat(gaze_added_mem)
    gaze_mem.loc[gaze_mem['Attention Level']=='Novel','av_x_coord']=0

    gaze_mem['gaze_towards'] = gaze_mem['av_x_coord'] - (59.8/2)

    gaze_mem.loc[(gaze_mem['Cued Side']=='<') 
                 & (gaze_mem['Attention Level'].isin(['Full', 'Side'])), 'gaze_towards'] = -gaze_mem[(gaze_mem['Cued Side']=='<') 
                                                                                                          & (gaze_mem['Attention Level'].isin(['Full', 'Side']))]['gaze_towards']

    gaze_mem.loc[(gaze_mem['Cued Side']=='>') 
                 & (gaze_mem['Attention Level'].isin(['None', 'Category'])), 'gaze_towards'] = -gaze_mem[(gaze_mem['Cued Side']=='>') & (gaze_mem['Attention Level'].isin(['None', 'Category'])), 'gaze_towards']

    gaze_mem_grp = gaze_mem.groupby(['Subject','Attention Level', 'Familiarity Rating','Trial Type'], as_index=False).mean()

    gaze_added_mem[label] = gaze_mem


TypeError: 'Series' objects are mutable, thus they cannot be hashed

# Plot

In [ ]:
for label in ['exp1','exp2']:
    
    gaze_mem_grp = gaze_added_mem[label].groupby(['Subject','Attention Level', 'Familiarity Rating','Trial Type'], as_index=False).mean()

    # scale by screen size
    gaze_mem_grp['scaled_gaze_towards'] = gaze_mem_grp['gaze_towards']/(59.8/2)

    # plot
    sb.set_style('white')

    f = sb.violinplot(x='Familiarity Rating', y='scaled_gaze_towards', 
                  data=gaze_mem_grp[gaze_mem_grp['Attention Level']!='Novel'], hue='Attention Level')

    plt.grid(False)
    plt.rc('xtick', labelsize=20)
    plt.rc('ytick', labelsize=20)
    plt.xlabel('Familiarity Rating',    fontsize = 20)
    plt.ylabel('Horizontal Gaze Drift', fontsize = 20)
    f.set(ylim=(-1, 1))
    plt.show()
    plt.savefig('Exp1_scaled_horiz_gaze.pdf')


# Regression

In [ ]:
# gaze_mem_grp.loc[(gaze_mem_grp['Attention Level']=='Novel'),'gaze_towards']=0
# gaze_mem_grp['FamiliarityRating'] = gaze_mem_grp['Familiarity Rating']
# gaze_mem_grp = gaze_mem_grp[gaze_mem_grp['Trial Type']=='Memory']
# d = []

# for subject in gaze_mem_grp['Subject'].unique(): 

#     data = gaze_mem_grp[(gaze_mem_grp['Subject']==subject)]
#     reg  = smf.ols(formula = "FamiliarityRating ~ gaze_towards", data = data).fit()
#     data = data.reset_index()
#     d.append(pd.concat([data, pd.DataFrame({'gaze_resid':reg.resid})], axis=1))
#     #print(data)
# gaze_reg = pd.concat(d)